<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/data-science-stack/polars-a-modern-dataframe-library.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<html>
<body>
    <div style="display: flex;">
        <a href="https://medium.com/@jgrygolec/polars-a-modern-dataframe-library-dcce43f43f3b" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: darkblue; font-weight: bold;">The notebook contains the code for Medium article</p>
            <p style="margin: 0; font-weight: bold; font-size: 1.5em; color: #3D4849;">Polars: A Modern DataFrame Library</p>
        </a>
        <a href="https://medium.com/@jgrygolec/polars-a-modern-dataframe-library-dcce43f43f3b" style="width: 15em; height: 15em;">
            <img src="https://cdn-images-1.medium.com/max/800/1*Oe1Jsf_23vUOma_2v6z75Q.png" style="width: 100%; height: 100%; object-fit: cover;">
        </a>
    </div>
</body>
</html>

# Project Setup

In [1]:
# Installing Python packages quietly

!pip install --quiet \
  httpx \
  pandas \
  plotly \
  polars \
  pyarrow \
  pympler \
  1> /dev/null

!pip install nbformat 1> /dev/null


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Importing the packages
from io import BytesIO
import os
from pathlib import Path
import time
from zipfile import ZipFile

import httpx
import pandas as pd
import polars as pl
import polars.selectors as cs
from pympler import asizeof

In [3]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables
COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
    from google.colab import userdata, data_table

    runtime_info = "Colab runtime"

    # Enabling Colab's data formatter for pandas
    data_table.enable_dataframe_formatter()
else:
    runtime_info = "Non Colab runtime"

print(runtime_info)

Non Colab runtime


In [4]:
# Download a zip file into the buffer

if not Path("./data/MetroPT3(AirCompressor).csv").exists():
    url = "https://archive.ics.uci.edu/static/public/791/metropt+3+dataset.zip"
    with httpx.Client() as client:
        response = client.get(url)
        buffer = BytesIO(response.content)

    # Unzip into the current directory
    with ZipFile(buffer) as zip_ref:
        # Extract all files to the current directory
        zip_ref.extractall("./data/")

In [5]:
# We need to load the data
df_pl = pl.read_csv("./data/MetroPT3(AirCompressor).csv")

# It is faster to convert to pandas from polars, than load with pandas
df_pd = df_pl.to_pandas()

In [6]:
df_pl.describe()

statistic,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.516948e6,"""1516948""",1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",7.584735e6,null,1.367826,8.984611,7.568155,0.055956,8.985233,62.644182,2.050171,0.836957,0.160611,0.919848,0.832664,0.00342,0.991437,0.904156,0.937107
"""std""",4.3791e6,null,3.25093,0.639095,3.3332,0.382402,0.638307,6.516261,2.302053,0.369405,0.367172,0.271528,0.373276,0.058381,0.092141,0.294378,0.242771
"""min""",0.0,"""2020-02-01 00:00:00""",-0.032,0.73,-0.036,-0.032,0.712,15.4,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",3.79237e6,null,-0.014,8.492,8.254,-0.022,8.494,57.775,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
"""50%""",7.58474e6,null,-0.012,8.96,8.784,-0.02,8.96,62.7,0.045,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
"""75%""",1.13771e7,null,-0.01,9.492,9.374,-0.018,9.492,67.25,3.8075,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
"""max""",1.516947e7,"""2020-09-01 03:59:50""",10.676,10.302,10.288,9.844,10.3,89.05,9.295,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# CPU Benchmarking

In [9]:
# Function to measure the time it takes to run a function, and store the results


def measure_time(func, trials, metadata, container, *args, **kwargs):
    # Measure the time it takes to run the function
    times = []
    for _ in range(trials):
        start_time = time.perf_counter()
        func(*args, **kwargs)
        end_time = time.perf_counter()
        times.append(end_time - start_time)

    interim = pl.DataFrame(
        {"time": times, "package": metadata["package"], "task": metadata["task"]}
    )
    container = pl.concat([container, interim])

    return container

In [10]:
# Defining prerequisites for the benchmarking
vars_of_interest = [
    "TP2",
    "TP3",
    "H1",
    "DV_pressure",
    "Reservoirs",
    "Oil_temperature",
    "Motor_current",
]
vars_of_interest_pd = {var: "mean" for var in vars_of_interest}

In [11]:
# Defining test functions

def read_csv_pl():
    return pl.read_csv("./data/MetroPT3(AirCompressor).csv")


def read_csv_pd():
    return pd.read_csv("./data/MetroPT3(AirCompressor).csv")


def describe_pl():
    return df_pl.describe()


def describe_pd():
    return df_pd.describe()


def filter_groupby_pl():
    return (
        df_pl.filter(pl.col("COMP") == pl.lit(1))
        .group_by("DV_eletric")
        .agg(cs.by_name(vars_of_interest).mean())
    )                   


def filter_groupby_pd():
    return df_pd[df_pd["COMP"] == 1].groupby("DV_eletric").agg(vars_of_interest_pd)



def sort_pl():
    return df_pl.sort(by="timestamp")


def sort_pd():
    return df_pd.sort_values("timestamp")

In [18]:
# Prepare the benchmarking container
benchmark = pl.DataFrame()

# Running the tests
benchmark = measure_time(
    func=describe_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "describe"},
)
benchmark = measure_time(
    func=describe_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "describe"},
)

benchmark = measure_time(
    func=read_csv_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "read_csv"},
)
benchmark = measure_time(
    func=read_csv_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "read_csv"},
)

benchmark = measure_time(
    func=filter_groupby_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "filter_groupby"},
)

benchmark = measure_time(
    func=filter_groupby_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "filter_groupby"},
)


benchmark = measure_time(
    func=sort_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "sort"},
)

benchmark = measure_time(
    func=sort_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "sort"},
)

In [86]:
from turtle import width
import plotly.express as px

# Group by package and task, calculate mean and standard error
df_grouped = benchmark.group_by(["package", "task"]).agg(
    mean_time=pl.mean("time"),
    std_error=pl.std("time"),
)


# Define the custom order
custom_order = ['read_csv', 'describe', 'filter_groupby', 'sort']
df_order = pl.DataFrame({
    'task': custom_order,
    'order': list(range(len(custom_order)))
})
df_grouped = df_grouped.join(df_order, on='task')
df_grouped = df_grouped.sort('order')
df_grouped = df_grouped.drop('order')

# Create the bar plot

fig = px.bar(
    df_grouped,
    x="task",
    y="mean_time",
    color="package",
    color_discrete_map={"pandas": "#e4048c", "polars": "#107cff"},
    barmode="group",
    #error_y="std_error",

)

fig.update_layout(
    {   "height": 630,
        "width": 1200,
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgb(37, 50, 59)",  # dark blue background
        "legend": {
            "font": {"color": "white", "size": 25},
            "title_text": "",
            "yanchor": "top",
            "y": 0.8,
            "xanchor": "center",
            "x": 0.5,
        },  # white legend
        "title": {
            "text": "Benchmarking Wall Time",
            "font": {"color": "white", "size": 40},  # white title
            "x": 0.5,  # center title
            "xanchor": "center",  # center title
        },
        "margin": {"l": 100, "r": 100, "t": 100, "b": 100},  # increase margins
    }
)

# Add text labels
fig.for_each_trace(lambda trace: trace.update(text = [f'{y:.2f}s' for y in trace.y], textposition='outside', textfont=dict(color='white', size=20)))


# Turn off grid lines and set axis labels and title to white
fig.update_xaxes(title_text="", showgrid=False, tickfont=dict(size=20, color='white'))
fig.update_yaxes(title_text="", showticklabels=False, showgrid=False, title_font=dict(size=25, color='white'), tickfont=dict(size=20, color='white'))

# Increase the y-axis range by 10% to create space for the label of the highest bar
max_y = df_grouped['mean_time'].max()
fig.update_yaxes(range=[0, max_y * 1.1])

# Remove lines around bars
fig.update_traces(marker_line_width=0, marker_line_color='rgba(0, 0, 0, 0)')


fig.show()

<html>
<body>
    <div style="display: flex;">
            <a href="https://medium.com/@jgrygolec" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Thank you for attention!</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849; font-weight: bold;">Click to see more of my articles on Medium.</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">BR,</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Jarek Grygolec</p>
        </a>
        <a href="https://medium.com/@jgrygolec" style="width: 30em; height: 15em; background-repeat: no-repeat; background-size: cover; background-position: center;">
                    <img src="https://miro.medium.com/v2/resize:fit:3992/0*WCQwsoZC0FA2-haq" style="width: 100%; height: 100%; object-fit: cover;"></a>
    </div>
</body>
</html>